In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

Test with poland

In [2]:
#file_name='minimal_seats-Poland_without91.xlsx'

In [3]:
file_name='minimal_seats-Australia.xlsx'

In [4]:

def country_df(file_name): 
    df=pd.read_excel(f'mvws/{file_name}')
    ###get country name part###
    first_split=file_name.split('-')
    country_name=first_split[1].split('.')[0]
    ### get dates from excel###
    dates=df['Key_1'].unique()
    dates_dt_1= pd.to_datetime(dates,errors='coerce',format='-%b-%y').sort_values()
    dates_dt_2= pd.to_datetime(dates,errors='coerce',format='%Y-%m').sort_values() #sometimes date format changes
    new_df=pd.DataFrame()
    if not dates_dt_2.empty:       ## if date format changed we need this convoluted bit here:
        dates_dt_2_1=dates_dt_2.strftime('%Y-%m')
        dates_dt_2_formatted=dates_dt_2.strftime('-%b-%y')

        sort_dates_str_1=dates_dt_1.strftime('-%b-%y')

        for i,date in enumerate(dates_dt_2_1):
            relevant_rows=df[df['Key_1']==date]
            parties=relevant_rows['Key_2'].tolist()
            mvws=relevant_rows['Value_1'].tolist()
            new_df[i]=pd.Series([parties,mvws])
            new_df.rename(columns={i:dates_dt_2_formatted[i]},inplace=True)

    ### create df as wanted
    for date in sort_dates_str_1:
        relevant_rows=df[df['Key_1']==date]
        parties=relevant_rows['Key_2'].tolist()
        mvws=relevant_rows['Value_1'].tolist()
        new_df[date]=pd.Series([parties,mvws])
    new_df = new_df.loc[:, new_df.columns.notna()]        
    new_df=new_df.add_prefix(f'{country_name}')
    return new_df

In [5]:
#poland_df=country_df(file_name)

In [6]:
all_mvw_files=[i for i in os.listdir('mvws') if i.endswith('.xlsx')]

In [7]:
country_dfs = []
for mvw_file in all_mvw_files: 
    df=country_df(mvw_file)
    country_dfs.append(df)
all_countries_df= pd.concat(country_dfs,axis=1)

In [8]:
indexnames= ['parties','mvws']
all_countries_df.index = indexnames

Now add the original seats to the df

In [9]:
def update_seats(file_name,all_countries_df): 
    df= pd.read_excel(f'prelims/{file_name}')
    dates=df['YearMonth'].unique()
    country_name=file_name.split('-')[1].split('.')[0]
    formatted_dates=[f"{country_name}{date}" for date in dates]
    
    for date in formatted_dates: 
        if date in all_countries_df.columns:
            relevant_rows= df[df['YearMonth']==date.replace(f'{country_name}','')] 
            parties_from_file= relevant_rows['Party'].tolist()
            parties_from_df=all_countries_df.loc['parties',date]
            if parties_from_df==parties_from_file: 
                if 'Seats' not in all_countries_df.index:
                    all_countries_df.loc['Seats']=[np.nan]*len(all_countries_df.columns)
                seats= relevant_rows['# of Seats'].tolist()
                all_countries_df.at['Seats',date]= seats
            else: all_countries_df.at['Seats', date] = 'error'
        else: print(f'Date {date} not found in final_df columns.')
    return all_countries_df

In [10]:
file_name='Preliminaries-Australia.xlsx'

In [11]:
df_2=update_seats(file_name,all_countries_df)

now add the power indices: 

In [12]:
file_name='power_indices-Australia.xlsx'

In [13]:
def update_powerindices(file_name,all_countries_df): 
    file=pd.ExcelFile(f'powerindices/{file_name}')
    for sheet_name in file.sheet_names: 
        df=pd.read_excel(file,sheet_name=sheet_name)
        country_name=file_name.split('-')[1].split('.')[0]
        date=f'{country_name}{sheet_name}'
        if date in all_countries_df: 
            parties_from_sheet=df['Party'].tolist()
            parties_from_df=all_countries_df.loc['parties',date]
            if parties_from_sheet==parties_from_df: 
                for col in df.columns[1:]: 
                    if col not in all_countries_df.index: 
                        all_countries_df.loc[col] = [np.nan] * len(all_countries_df.columns)    
                    all_countries_df.at[col, date] = df[col].tolist()
            else: all_countries_df.at[col, date] = 'error' 
        else: print(f'Date {date} not found in final_df columns.')
    return all_countries_df

In [14]:
df3=update_powerindices(file_name,all_countries_df)

In [15]:
all_originalseats_files=[i for i in os.listdir('prelims') if i.endswith('.xlsx')]

In [16]:
for prelimfile in all_originalseats_files: 
    all_countries_df=update_seats(prelimfile,all_countries_df)

Date Israel_99_modified1992-06 not found in final_df columns.
Date Israel_99_modified2003-01 not found in final_df columns.
Date Israel_99_modified2009-02 not found in final_df columns.
Date Israel_99_modified2013-01 not found in final_df columns.
Date Israel_99_modified2019-04 not found in final_df columns.
Date Israel_99_modified2019-09 not found in final_df columns.
Date Krohn2001 not found in final_df columns.
Date Krohn2002 not found in final_df columns.
Date Krohn2003 not found in final_df columns.
Date Krohn2004 not found in final_df columns.
Date Krohn2005 not found in final_df columns.
Date Krohn2006 not found in final_df columns.
Date krohn_example2001 not found in final_df columns.
Date krohn_example2002 not found in final_df columns.
Date krohn_example2003 not found in final_df columns.
Date krohn_example2004 not found in final_df columns.
Date krohn_example2005 not found in final_df columns.
Date krohn_example2006 not found in final_df columns.
Date Poland_without911993-09

In [17]:
all_powerfiles=[i for i in os.listdir('powerindices') if i.endswith('.xlsx')]

In [18]:
for powerfile in all_powerfiles: 
    all_countries_df=update_powerindices(powerfile,all_countries_df)

Date Israel_99_modified1992-06 not found in final_df columns.
Date Israel_99_modified2003-01 not found in final_df columns.
Date Israel_99_modified2009-02 not found in final_df columns.
Date Israel_99_modified2013-01 not found in final_df columns.
Date Israel_99_modified2019-04 not found in final_df columns.
Date Israel_99_modified2019-09 not found in final_df columns.
Date Krohn2001 not found in final_df columns.
Date Krohn2002 not found in final_df columns.
Date Krohn2003 not found in final_df columns.
Date Krohn2004 not found in final_df columns.
Date Krohn2005 not found in final_df columns.
Date Krohn2006 not found in final_df columns.
Date Poland_without911993-09 not found in final_df columns.
Date Poland_without911997-09 not found in final_df columns.
Date Poland_without912001-09 not found in final_df columns.
Date Poland_without912005-09 not found in final_df columns.
